purpose: filter out similar images

In [ ]:
from PIL import Image
import imagehash
import glob
import ntpath
import os

In [ ]:
def get_files_dict(path, buf_size):
    files = []
    for file in glob.glob(f"{path}/*"):
        files.append(file)
    files = sorted(files)
    files_dict = {}
    for i in range(len(files)):
        start = i+1
        end = i+buf_size
        if end >= len(files):
            end = len(files) - 1

        files_dict[files[i]] = files[start:end]

    return files_dict

### Global vars

In [ ]:
NUM_RUNS = 3
BUF_SIZE = 10

## Make dirs 

In [ ]:
for i in range(1, NUM_RUNS+1):
    dirname = f"run{i}"
    if os.path.exists(dirname):
        os.removedirs(dirname)
    os.makedirs(dirname)

## Load file paths and store in dict

In [ ]:
input_dir = ""
dirs = [input_dir]
for i in range(1, NUM_RUNS+1):
    dirs.append(f"run{i}")

In [ ]:
cutoff = 10

In [ ]:
for i in range(len(dirs)-1):
    _in = dirs[i]
    _out = dirs[i+1]
    
    files_dict = get_files_dict(_in, BUF_SIZE)
    
    similars = []
    for f1 in files_dict:
        if f1 in similars:
            continue

        img1 = Image.open(f1)
        hash1 = imagehash.average_hash(img1)

        img1.save("{}/{}".format(_out, ntpath.basename(f1)))

        for f2 in files_dict[f1]:
            img2 = Image.open(f2)
            hash2 = imagehash.average_hash(img2)

            if hash1-hash2 < 12:
                similars.append(f2)
        
        print("RUN {} - similars: {}".format(i+1, len(similars)), end='\r')
